colab - https://colab.research.google.com/drive/1qBBkhh_p3F67xK2ezcurmjDuEsFwh48g?authuser=0#scrollTo=-Dwa_X-VVkft

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
!pip install -q dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 10.1 MB/s eta 0:00:00


In [3]:
import dspy

lm = dspy.LM(
    model="openai/gpt-4.1-mini",
    temperature=1,
    api_key=OPENAI_API_KEY
)
dspy.configure(lm=lm)

In [4]:
lm("hi")

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Hello! H...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


['Hello! How can I assist you today?']

In [7]:
import dspy
from datasets import load_dataset

def dataset():
  train = load_dataset("AI-MO/aimo-validation-aime")["train"]
  train = [
      dspy.Example({
          "problem": x["problem"],
          "solution": x["solution"],
          "answer": x["answer"]
      }).with_inputs("problem")
      for x in train
  ]

  import random
  random.Random(0).shuffle(train)
  train_len = len(train)

  test = load_dataset("MathArena/aime_2025")["train"]
  test = [
      dspy.Example({
          "problem": x["problem"],
          "answer": x["answer"]
      }).with_inputs("problem")
      for x in test
  ]

  train_set = train[:int(0.5*train_len)]
  val_set = train[int(0.5*train_len):]
  test_set = test*5

  return train_set, val_set, test_set



# quick notes
- we are using 2 different datasets for train and test (both related to math)
- we use a validation set from train to imrprove optimisation
- .with_inputs("problem") tells dspy to only send problem to llm and hide the solution / answer
- train and val sets are half of the og train dataset
- test set is duplicated to check llm performance across 5 different responses of same query

In [8]:
train_set, val_set, test_set = dataset()
len(train_set), len(val_set), len(test_set)

(45, 45, 150)

In [9]:
print("Problem:\n")
print(train_set[0]['problem'])
print("\n\nSolution:\n")
print(train_set[0]['solution'])
print("\n\nAnswer:\n")
print(train_set[0]['answer'])

Problem:

In isosceles trapezoid $ABCD$, parallel bases $\overline{AB}$ and $\overline{CD}$ have lengths $500$ and $650$, respectively, and $AD=BC=333$. The angle bisectors of $\angle{A}$ and $\angle{D}$ meet at $P$, and the angle bisectors of $\angle{B}$ and $\angle{C}$ meet at $Q$. Find $PQ$.


Solution:

We have the following diagram:

Let $X$ and $W$ be the points where $AP$ and $BQ$ extend to meet $CD$, and $YZ$ be the height of $\triangle AZB$. As proven in Solution 2, triangles $APD$ and $DPW$ are congruent right triangles. Therefore, $AD = DW = 333$. We can apply this logic to triangles $BCQ$ and $XCQ$ as well, giving us $BC = CX = 333$. Since $CD = 650$, $XW = DW + CX - CD = 16$.
Additionally, we can see that $\triangle XZW$ is similar to $\triangle PQZ$ and $\triangle AZB$. We know that $\frac{XW}{AB} = \frac{16}{500}$. So, we can say that the height of the triangle $AZB$ is $500u$ while the height of the triangle $XZW$ is $16u$. After that, we can figure out the distance fro

In [16]:
class GenerateResponse(dspy.Signature):
  """Solve the problem and provide the answer in the correct format."""
  problem = dspy.InputField()
  answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

In [11]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
  correct_answer=int(example["answer"])
  try:
    llm_answer = int(prediction.answer)
  except ValueError as e:
    return 0
  return int(correct_answer == llm_answer)

In [12]:
import dspy

evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 0.00 / 2 (0.0%):   1%|          | 1/150 [00:17<43:34, 17.54s/it]

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 1.00 / 3 (33.3%):   2%|▏         | 3/150 [00:23<16:26,  6.71s/it]

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 67.00 / 150 (44.7%): 100%|██████████| 150/150 [05:18<00:00,  2.12s/it]

2026/01/16 20:54:50 INFO dspy.evaluate.evaluate: Average Metric: 67 / 150 (44.7%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, let's understand the problem in detail: We are given two nu...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"First, let's analyze the given configuration and the notation care...",588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players, each chooses one flavor: chocolate (C), vanilla...",764,✔️ [0]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We need to find the number of integer pairs \((x,y)\) with \(-100 ...",117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are asked to consider all 8-digit positive integers that use ea...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We have a regular 24-gon with vertices \( S = \{v_0, v_1, \ldots, ...",4208,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,Let's analyze the problem step-by-step. We have an 11-sided polygo...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,We have the sequence defined by \[ x_1 = \frac{25}{11} \] and \[ x...,336,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,"Given \(\triangle ABC\) with \(\angle A = 90^\circ\), and \(BC = 3...",\[\n\boxed{104}\n\],✔️ [0]


EvaluationResult(score=44.67, results=<list of 150 results>)

workflow so far
- create testset -> array of dspy.examples (configure input to "problem" for each example)
- setup program schema (configure input, output with dspy format and set high level prompt)
- configure metric to define output vals to 0 or 1
- setup evaluate()
- run


quick notes
- we define a signature class to add a high level prompt - this provides better context to the llm
- ensure that the variables (question, solution) in signature class are the same as the ones in the dataset
- a simpler way to write signature is dspy.chainofthough("problem -> solution") but it doesnt provide much context


# optimize with GEPA
# GEPA is reflective prompt optimizer, it reflects on its thinking wrt to solutions to frame better prompt

In [14]:
def metric_w_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
  correct_answer = int(example["answer"])
  correct_solution = example.get("solution", "")
  try:
    llm_answer = int(prediction.answer)
  except ValueError as e:
    feedback = f"""The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}',
    which couldn't be parsed as a python integer.
    Please ensure your answer is a valid integer without any additional text or formatting. The correct answer is '{correct_answer}'"""
    if correct_solution:
      feedback += f""" Here's the full step-by-step solution:\n{correct_solution}\n\n
      Think about what takeaways you can learn from this solution to improve your future answers and
      approach to similar problems and ensure your final answer is a valid integer."""
    return dspy.Prediction(score = 0, feedback=feedback)

  score = int(correct_answer == llm_answer)

  feedback = ""

  if score == 1:
    feedback = f"Your answer is correct. The correct answer is '{correct_answer}'."
  else:
    feedback = f"Your answer is incorrect. The correct answer is '{correct_answer}'."

  if correct_solution:
    feedback += f""" Here's the full step-by-step solution:\n{correct_solution}\n\n
    Think about what takeaways you can learn from this solution to improve your future answers
    and approach to similar problems."""


  return dspy.Prediction(score=score, feedback=feedback)


quick notes

- we create a new metric that feeds feedback into the llm thinking
- first, we check for any errors in output. if yes, we feeback a string referring correct answer to predicted along with solution (if any)
- next, we check if score is 1 (correct answer) and let it know that it was correct along with solution (if any)
- if score is 0 (wrong answer), we provide the correct answer with reasoning from solutino
- then we return dspy.prediction with updated score and feedback text

In [20]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_w_feedback,
    num_threads=32,
    auto="light",
    track_stats=True,
    reflection_minibatch_size=3,
    reflection_lm=dspy.LM(model="gpt-5", temperature=1.0, max_tokens=32000, api_key=OPENAI_API_KEY)
)


optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set
  )

2026/01/16 22:51:22 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 560 metric calls of the program. This amounts to 6.22 full evals on the train+val set.
2026/01/16 22:51:22 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/560 [00:00<?, ?rollouts/s]/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [01:07<00:00, 22.64s/it]

2026/01/16 22:54:30 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='```\nYou...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2026/01/16 22:55:25 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You are given a single field named "problem" containing a standalone math/number theory/combinatorics problem. Your job is to solve it correctly and output the final result in the required concise format.

Follow these guidelines:

1) Output format
- Provide two sections in order:
  - reaso

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:53<00:00, 17.70s/it]

...

2026/01/17 00:10:40 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


2026/01/17 00:12:08 INFO dspy.teleprompt.gepa.gepa: Iteration 16: Proposed new text for predict: You are given a single field named "problem" containing a standalone math/number theory/combinatorics (and occasionally geometry) problem. Your job is to solve it correctly and output the final result in the required concise format.

Follow these guidelines:

1) Output format
- Provide two sections in order:
  - reasoning: a concise but complete derivation
  - answer: the final result only (no units, labels, punctuation, leading zeros, or extra text)
- The final answer must be a single value or expression exactly as the problem requests.

2) General solving strategy
- Parse the problem carefully and restate key constraints internally.
- Prefer exact algebraic manipulation over numeric approximation. Keep fractions reduced; simplify radicals. Double-check arithmetic and algebraic transformations.
- Prove uniqueness or extremality (greatest/smallest) rather than stopping at a candidate. If mu

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:30<00:00, 10.06s/it]

2026/01/17 00:16:17 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


2026/01/17 00:17:24 INFO dspy.teleprompt.gepa.gepa: Iteration 17: Proposed new text for predict: You are given a single field named "problem" containing one standalone math/number theory/combinatorics/geometry problem. Your job is to solve it correctly and output the final result in the required concise format.

Output format
- Provide two sections in order:
  - reasoning: a concise but complete derivation
  - answer: the final result only (no units, labels, punctuation, or extra text)
- The final answer must be a single value or expression exactly as the problem requests.

General solving strategy
- Parse the problem carefully and restate key constraints internally (ranges, integrality, base/digit bounds, geometric configurations).
- Prefer exact symbolic/algebraic manipulation over numeric approximation. Keep fractions exact and rationalize when needed.
- Prove uniqueness or extremality (greatest/smallest) rather than stopping at a candidate. If multiple solutions are possible, count

In [21]:
program.predict.signature.instructions

'Solve the problem and provide the answer in the correct format.'

In [22]:
optimized_program.predict.signature.instructions

'You are given a single field named "problem" containing a standalone math/number theory/combinatorics problem. Your job is to solve it correctly and output the final result in the required concise format.\n\nFollow these guidelines:\n\n1) Output format\n- Provide two sections in order:\n  - reasoning: a concise but complete derivation\n  - answer: the final result only (no units, labels, punctuation, or extra text)\n- The final answer must be a single value or expression exactly as the problem requests.\n\n2) General solving strategy\n- Parse the problem carefully and restate key constraints internally.\n- Prefer exact algebraic manipulation over numeric approximation. Double-check arithmetic.\n- Prove uniqueness or extremality (greatest/smallest) rather than stopping at a candidate. If multiple solutions are possible, count them correctly.\n\n3) Useful domain tactics and checks\n- Base representation and digit constraints:\n  - In base-k, every digit must be in {0, 1, ..., k-1}. Lead

In [23]:
evaluate(optimized_program)

Average Metric: 1.00 / 1 (100.0%):   1%|          | 1/150 [00:05<12:38,  5.09s/it]

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 5.00 / 5 (100.0%):   3%|▎         | 4/150 [00:10<04:48,  1.98s/it]

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 69.00 / 150 (46.0%): 100%|██████████| 150/150 [03:19<00:00,  1.33s/it]

2026/01/17 00:34:25 INFO dspy.evaluate.evaluate: Average Metric: 69 / 150 (46.0%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, interpret the given numbers in base \( b \): - \(17_b = 1 \...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"First, we set coordinate axes and place points to utilize given di...",588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players, each choosing a flavor from {Chocolate (C), Van...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We are given the equation \[ 12x^2 - xy - 6y^2 = 0, \] with intege...",117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are looking at the set of all 8-digit numbers that use each dig...,-297,✔️ [0]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertices \( S \), and we want t...",4464,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-sided polygon \( A_1 A_2 \cdots A_{11} \) with the f...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,"Given the sequence: \[ x_1 = \frac{25}{11}, \quad x_{k+1} = \frac{...",108,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given right triangle \( \triangle ABC \) with \(\angle A = 90^\cir...,104,✔️ [1]


EvaluationResult(score=46.0, results=<list of 150 results>)

In [24]:
evaluate(optimized_program)

Average Metric: 65.00 / 150 (43.3%): 100%|██████████| 150/150 [00:00<00:00, 173.31it/s]


2026/01/17 00:37:31 INFO dspy.evaluate.evaluate: Average Metric: 65 / 150 (43.3%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, translate the given numbers from base \( b \) to decimal ex...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"First, we set coordinate axes and place points to utilize given di...",588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players choosing among 3 flavors: chocolate (C), vanilla...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We want the integer ordered pairs \((x,y)\) with \(-100 \leq x,y \...",119,✔️ [0]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We consider the 8-digit integers formed by the digits 1 through 8 ...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertices \( S \), and we want t...",4464,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-gon \( A_1 A_2 \ldots A_{11} \) with the following p...,64,✔️ [0]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,Given the recurrence: \[ x_{k+1} = \frac{1}{3}\left(x_k + \frac{1}...,722,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given: \(\triangle ABC\) is right-angled at \(A\) with \(BC = 38\)...,98,✔️ [0]


EvaluationResult(score=43.33, results=<list of 150 results>)

In [25]:
evaluate(optimized_program)

Average Metric: 65.00 / 150 (43.3%): 100%|██████████| 150/150 [00:00<00:00, 157.71it/s]

2026/01/17 00:40:08 INFO dspy.evaluate.evaluate: Average Metric: 65 / 150 (43.3%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, translate the given numbers from base \( b \) to decimal ex...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"First, we set coordinate axes and place points to utilize given di...",588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players choosing among 3 flavors: chocolate (C), vanilla...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We want the integer ordered pairs \((x,y)\) with \(-100 \leq x,y \...",119,✔️ [0]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We consider the 8-digit integers formed by the digits 1 through 8 ...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertices \( S \), and we want t...",4464,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-gon \( A_1 A_2 \ldots A_{11} \) with the following p...,64,✔️ [0]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,Given the recurrence: \[ x_{k+1} = \frac{1}{3}\left(x_k + \frac{1}...,722,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given: \(\triangle ABC\) is right-angled at \(A\) with \(BC = 38\)...,98,✔️ [0]


EvaluationResult(score=43.33, results=<list of 150 results>)

In [26]:
with dspy.context(cache=False):
    evaluate(optimized_program)

Average Metric: 65.00 / 150 (43.3%): 100%|██████████| 150/150 [00:04<00:00, 30.79it/s]

2026/01/17 00:48:46 INFO dspy.evaluate.evaluate: Average Metric: 65 / 150 (43.3%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,"First, translate the given numbers from base \( b \) to decimal ex...",70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"First, we set coordinate axes and place points to utilize given di...",588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players choosing among 3 flavors: chocolate (C), vanilla...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We want the integer ordered pairs \((x,y)\) with \(-100 \leq x,y \...",119,✔️ [0]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We consider the 8-digit integers formed by the digits 1 through 8 ...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertices \( S \), and we want t...",4464,✔️ [0]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-gon \( A_1 A_2 \ldots A_{11} \) with the following p...,64,✔️ [0]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,Given the recurrence: \[ x_{k+1} = \frac{1}{3}\left(x_k + \frac{1}...,722,✔️ [0]
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given: \(\triangle ABC\) is right-angled at \(A\) with \(BC = 38\)...,98,✔️ [0]


Final thoughts

there was a slight improvement in performance from 44.67 to 46

this is not great but could also be because
- the dataset is really small and less diverse
- the llm is not the best and prompting can only do so much


Cons
- the optimisation process took > 60 min for a ~1% increase in performance


finally, I for this specific experiment, I didnt think it was worth the tokens for a 2% imrprovement but according to gepa results in their docs, it seems really useful

next time
- Ill consider better lengthy dataset so it has diverse problems during training
- or consider a smaller dataset that contains all the diverse topics in the dataset